In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from sklearn.metrics import classification_report, confusion_matrix


In [ ]:
# Define paths to your dataset
train_data_dir = 'path_to_train_dataset'
validation_data_dir = 'path_to_validation_dataset'


In [ ]:
# Data preprocessing
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    # Other augmentation parameters
)
validation_datagen = ImageDataGenerator(
    rescale=1.0/255,
)
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'  # or 'categorical' if you have multiple classes
)

validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'  # or 'categorical' if you have multiple classes
)

In [ ]:
# Create the base model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = base_model.output
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)  # Change to number of classes if more than 2
model = Model(inputs=base_model.input, outputs=predictions)


In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])



In [ ]:
# Train the model
model.fit(train_generator, epochs=10, validation_data=validation_generator)



In [ ]:
# Evaluate the model
validation_loss, validation_accuracy = model.evaluate(validation_generator)
print("Validation Loss:", validation_loss)
print("Validation Accuracy:", validation_accuracy)



In [ ]:
# Make predictions
predictions = model.predict(validation_generator)
predicted_labels = np.round(predictions)  # Adjust threshold based on your problem



In [ ]:
# Calculate metrics
true_labels = validation_generator.classes
confusion = confusion_matrix(true_labels, predicted_labels)
classification_rep = classification_report(true_labels, predicted_labels)



In [ ]:
print("Confusion Matrix:")
print(confusion)
print("Classification Report:")
print(classification_rep)
